In [1]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.7 MB/s eta 0:00:00


In [2]:
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense
from keras import utils
from keras_tuner.tuners import RandomSearch, Hyperband, BayesianOptimization
import numpy as np

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train / 255
x_test = x_test / 255
y_train = utils.to_categorical(y_train, 10)
y_test = utils.to_categorical(y_test, 10)

4422102/4422102 [==============================] - 0s 0us/step


In [3]:
def build_model(hp):
    model = Sequential()
    activation_choice = hp.Choice('activation', values=['relu', 'sigmoid', 'tanh', 'elu', 'selu'])
    model.add(Dense(units=hp.Int('units_input',    # Полносвязный слой с разным количеством нейронов
                                   min_value=512,    # минимальное количество нейронов - 128
                                   max_value=1024,   # максимальное количество - 1024
                                   step=32),
                    input_dim=784,
                    activation=activation_choice))
    for i in range(hp.Int('num_hidden_layers', 4, 10)):
        model.add(Dense(units=hp.Int(f'units_hidden_{i}',
                           min_value=128,
                           max_value=1024,
                           step=32),
              input_dim=784,
              activation=activation_choice))
    model.add(Dense(10, activation='softmax'))
    model.compile(
        optimizer=hp.Choice('optimizer', values=['adam','rmsprop','SGD']),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

In [4]:
tuner = RandomSearch(
    build_model,                 # функция создания модели
    objective='val_accuracy',    # метрика, которую нужно оптимизировать -
                                 # доля правильных ответов на проверочном наборе данных
    max_trials=10,               # максимальное количество запусков обучения
    directory='test_directory'   # каталог, куда сохраняются обученные сети
    )
tuner.search_space_summary()

Search space summary
Default search space size: 8
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid', 'tanh', 'elu', 'selu'], 'ordered': False}
units_input (Int)
{'default': None, 'conditions': [], 'min_value': 512, 'max_value': 1024, 'step': 32, 'sampling': 'linear'}
num_hidden_layers (Int)
{'default': None, 'conditions': [], 'min_value': 4, 'max_value': 10, 'step': 1, 'sampling': 'linear'}
units_hidden_0 (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 1024, 'step': 32, 'sampling': 'linear'}
units_hidden_1 (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 1024, 'step': 32, 'sampling': 'linear'}
units_hidden_2 (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 1024, 'step': 32, 'sampling': 'linear'}
units_hidden_3 (Int)
{'default': None, 'conditions': [], 'min_value': 128, 'max_value': 1024, 'step': 32, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 

In [6]:
tuner.search(x_train,                  # Данные для обучения
             y_train,                  # Правильные ответы
             batch_size=256,           # Размер мини-выборки
             epochs=10,                # Количество эпох обучения
             validation_split=0.2,     # Часть данных, которая будет использоваться для проверки
             verbose=1
             )

Trial 10 Complete [00h 02m 41s]
val_accuracy: 0.10266666859388351

Best val_accuracy So Far: 0.8790000081062317
Total elapsed time: 00h 38m 49s


In [7]:
tuner.results_summary()

Results summary
Results in test_directory/untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 07 summary
Hyperparameters:
activation: elu
units_input: 672
num_hidden_layers: 5
units_hidden_0: 960
units_hidden_1: 416
units_hidden_2: 960
units_hidden_3: 992
optimizer: adam
units_hidden_4: 416
units_hidden_5: 864
units_hidden_6: 480
units_hidden_7: 704
units_hidden_8: 416
Score: 0.8790000081062317

Trial 04 summary
Hyperparameters:
activation: relu
units_input: 832
num_hidden_layers: 8
units_hidden_0: 800
units_hidden_1: 288
units_hidden_2: 512
units_hidden_3: 992
optimizer: rmsprop
units_hidden_4: 160
units_hidden_5: 256
units_hidden_6: 864
units_hidden_7: 704
units_hidden_8: 864
Score: 0.8754166960716248

Trial 03 summary
Hyperparameters:
activation: tanh
units_input: 608
num_hidden_layers: 9
units_hidden_0: 928
units_hidden_1: 512
units_hidden_2: 640
units_hidden_3: 224
optimizer: adam
units_hidden_4: 640
units_hidden_5: 160
units_hidden_6: 10

In [8]:
models = tuner.get_best_models(num_models=3)

In [9]:
for model in models:
  model.summary()
  model.evaluate(x_test, y_test)
  print()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 672)               527520    
                                                                 
 dense_1 (Dense)             (None, 960)               646080    
                                                                 
 dense_2 (Dense)             (None, 416)               399776    
                                                                 
 dense_3 (Dense)             (None, 960)               400320    
                                                                 
 dense_4 (Dense)             (None, 992)               953312    
                                                                 
 dense_5 (Dense)             (None, 416)               413088    
                                                                 
 dense_6 (Dense)             (None, 10)                4